# IPython Notebook Template for Visualizing Product Space

In [1]:
import sys
sys.path.append("./modules")
import d3plus
import pandas as pd
from linnaeus import classification #linnaeus requires cloning classifications.git 

## Choose the file for visualization

In [2]:
#Define which file to load 
#For example, Salina_Cruz_ps.csv or Ciudad_Ixtepec_ps.csv
#filename="sourcedata/Salina_Cruz_ps.csv"
filename="sourcedata/Ciudad_Ixtepec_ps.csv"

In [3]:
#Load into pd.dataframe
countryMatrix = pd.read_csv(filename)
countryMatrix = countryMatrix.ix[:,["hs4","M"]]
countryMatrix.head(2) 

,hs4,M
0,101,0
1,102,0


## Set up color map and code table

In [4]:
#Convert RGB values to HEX values
def hexify(a):
    return "%02#x" % int(a)

In [5]:
#Load up a color mapping
colorsForHS4 = pd.read_csv("sourcedata/hs4_colors_int_atlas.csv")
colorsForHS4 = colorsForHS4.ix[:,["hs4","atlas_color"]]

In [6]:
#Load up a product code table, getting only the 4digit level entries
product_classification = classification.load("product/HS/Atlas/out/hs92_atlas.csv")
productCodes = product_classification.table[product_classification.table['level']=='4digit']
productCodes.columns.values[0] = "code" #Rename the first column to 'code'
productCodes.head(2)

,code,name,level,parent_id
114,0101,"Live horses, asses, mules or hinnies",4digit,16
115,0102,Live bovine animals,4digit,16


In [7]:
#Cleaned up HS92 4digit Product Codes
countryMatrix["hs4"] = countryMatrix["hs4"].astype(int).astype(str).str.zfill(4)
colorsForHS4["hs4"] = colorsForHS4["hs4"].astype(int).astype(str).str.zfill(4)

In [8]:
#Merging all three DataFrames into one DataFrane, countryProducts
countryMatrix = countryMatrix.merge(colorsForHS4,how='left',left_on='hs4',right_on='hs4')
countryProducts = countryMatrix.merge(productCodes,how='left',left_on='hs4',right_on='code')
countryProducts.head(2)

,hs4,M,atlas_color,code,name,level,parent_id
0,0101,0,"(255, 226, 175)",0101,"Live horses, asses, mules or hinnies",4digit,16
1,0102,0,"(255, 226, 175)",0102,Live bovine animals,4digit,16


In [9]:
#We have rgb values for atlas_color, but d3plus wants hex:
#Extract the rgb values, and convert them into a hex color column 
atlasColorFragments = countryProducts["atlas_color"].str.extract('(\d*), (\d*), (\d*)')

In [10]:
#A function to convery rgb values to hex
def hexify(a):
    return "%02x" % int(a)

In [11]:
countryProducts["hex"] = "#"+atlasColorFragments[0].apply(hexify)+atlasColorFragments[1].apply(hexify)+atlasColorFragments[2].apply(hexify)

In [12]:
#Adding a column for Spanish translation (MEX)
df_es=pd.read_csv('sourcedata/products_mexico_prospedia.csv')
df_es = df_es[df_es['level']=='4digit'].ix[:,["code","name_es"]]
df_es["code"] = df_es["code"].astype(int).astype(str).str.zfill(4)
countryProducts_withEs = countryProducts.merge(df_es,how='left',left_on='code',right_on='code')
countryProducts_withEs.head(2)

,hs4,M,atlas_color,code,name,level,parent_id,hex,name_es
0,0101,0,"(255, 226, 175)",0101,"Live horses, asses, mules or hinnies",4digit,16,#ffe2af,"Caballos, asnos, mulos y burdéganos, vivos."
1,0102,0,"(255, 226, 175)",0102,Live bovine animals,4digit,16,#ffe2af,Animales vivos de la especie bovina.


## Now we're almost ready to create our product space visualization!

`id`: set to `hs4` for the product code <br />
`color`: set to the column name `hex` for the hex color values <br />
`name`: set to column name `name` to show the text; set to `name_es` for Spanish

In [13]:
#First some configuration to set which columns the visualization should grab from our dataframe
productspace = d3plus.ProductSpace(id='hs4',color='hex',name='name_es')

In [14]:
#Product space of Ixtepec: What export products does Ciudad Ixtepec export above their global share?
productspace.draw(countryProducts_withEs)

<IPython.core.display.Javascript object>